# Generate Transition Plots
This notebooks saves a whole history of generative reverse diffusion for solar min and solar max casas, and saves a field line plot for each step. Fixed seeds are used for reproducability.

In [1]:
import os
import sys

sys.path.append('..')
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

import torch
import numpy as np
import random
import pylab as plt
import tqdm

from coronal_diffusion import dataset
from coronal_diffusion import visualization_tools as vt
from coronal_diffusion import sampler

import config

/home/dedasilv/.micromamba/envs/coronal-diffusion-modeling/lib/python3.11/site-packages/ai/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
def set_seeds(seed):
    torch.use_deterministic_algorithms(True)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [3]:
os.chdir('..')
print('Current directory is:', os.getcwd())

Current directory is: /home/dedasilv/coronal-diffusion-modeling


In [4]:
plot_dir = 'plots/transition/'
weights_file = "checkpoints/experiment14-refactor-alldata_2.pth"
tasks = [(0, 'min', config.seed_helper_min, 829),
         (1, 'max', config.seed_helper_max, 829)]

!rm -rf $plot_dir
!mkdir -p $plot_dir

In [5]:
saved_history = {}

for radio_flux, tag, seed_helper, seed in tasks:
    set_seeds(seed)

    saved_history[tag] = sampler.sample(
        weights_file=weights_file,
        radio_flux=radio_flux,
        method='ddim',
        seed_helper=seed_helper,
        return_history=True
    )

In [6]:
len(saved_history['max'])

21

In [7]:
for tag, history in saved_history.items():
    for step, (G, H) in enumerate(tqdm.tqdm(history)):
        percent = int(100 * step / (len(history) - 1))

        #if percent % 25 != 0:
        #    continue
            
        sh = vt.SHVisualizer(G, H)
        fig = sh.visualize_field_lines(r=1.1, grid_density=40)

        fig.update_layout(
            title=dict(text=f"{percent}%", font=dict(size=50), automargin=True, yref='paper')
        )
        fig.write_image(f'{plot_dir}/{tag}_percent{percent:03d}_step{step}.png', width=800, height=600)
        fig.data = []

100%|████████████████████████████████████████| 21/21 [11:29<00:00, 32.84s/it]
